This notebook was designed to run under the assumption that shared drive folders for the project are mounted.

#Setup

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
!pip install kaggle
!pip install pyspark

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import os, time
from google.colab import drive, files
from pyspark import SparkContext
from pyspark.sql import SQLContext
from pyspark.sql import SparkSession
import re

In [ ]:
os.environ["TZ"] = "US/Eastern"
time.tzset()
drive.mount('/content/drive')

#Authenticating with Kaggle using kaggle.json
Navigate to https://www.kaggle.com. Then go to the Account tab of your user profile and select Create API Token. This will trigger the download of kaggle.json, a file containing your API credentials.

Then run the cell below to upload kaggle.json to your Colab runtime.

In [ ]:
uploaded = files.upload()

for fn in uploaded.keys():
  print('User uploaded file "{name}" with length {length} bytes'.format(
      name=fn, length=len(uploaded[fn])))
  
# Then move kaggle.json into the folder where the API expects to find it.
!mkdir -p ~/.kaggle/ && mv kaggle.json ~/.kaggle/ && chmod 600 ~/.kaggle/kaggle.json

In [ ]:
!kaggle datasets download "rexhaif/emojifydata-en"

In [ ]:
!mkdir "kaggle"
!mkdir "kaggle/input"
!unzip emojifydata-en.zip -d "kaggle/input"

# Cleaning Dataset

In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
ss = SparkSession.builder \
    .config("spark.driver.memory", "10g") \
    .getOrCreate()
sc = ss.sparkContext
s  = SQLContext(sc)

Read _.txt as RDD. Change to other files

In [ ]:
tweets = sc.textFile("./kaggle/input/train.txt")

Convert to python list to concatenate words into tweets

In [ ]:
tweets_clean = tweets.flatMap(lambda x: x.split(' ')).filter(lambda x:x!='O').filter(lambda x:x!='').filter(lambda x:x!='<STOP>')
tweet_list = tweets_clean.collect()

In [ ]:
tweets = []
global counter
counter = -1

def wordsToTweets(x):
    global counter
    if(x=='<START>'):
        counter+=1
        tweets.append('')
    else:
        tweets[counter]+=(x+" ")

In [ ]:
for word in tweet_list:
    wordsToTweets(word)
# conver back to rdd
tweet_rdd = sc.parallelize(tweets)
tweets = tweet_rdd.zipWithIndex().map(lambda x: (x[1],x[0]))
# get emoji out of text
text_rdd = tweets.map(lambda x: (x[0],re.sub(":.*?:","",x[1])))
emoji_rdd = tweets.map(lambda x: (x[0],re.findall(":.*?:",x[1])))
emoji_rdd1 = emoji_rdd.map(lambda x:(x[0],x[1][0]))
max_emoji = emoji_rdd.map(lambda x: len(x[1])).max()
# join text with emoji rdd
for i in range(1,max_emoji):
    emoji_rdd2 = emoji_rdd.filter(lambda x:len(x[1])>i).map(lambda x: (x[0],x[1][i]))
    if i==1:
        emoji_rdd3 = emoji_rdd1.union(emoji_rdd2)
    else:
        emoji_rdd3 = emoji_rdd3.union(emoji_rdd2)

rdd_for_df = text_rdd.leftOuterJoin(emoji_rdd3).map(lambda x:(x[1][0][:-1],x[1][1][1:-1]))
# create dataframe
df = s.createDataFrame(rdd_for_df, ['text','emoji']).distinct()
df.show()

In [ ]:
df.write.csv('train')

In [ ]:
!zip -r train.zip train

In [ ]:
!mv ./train.zip ./drive/Shareddrives/EECS\ 545/